In [1]:
import numpy as np

In [172]:
class Robot(object):
    def __init__(self, initial_pos, r_vel_furrow, r_vel_head):
        self.cur_pos = initial_pos # m*2
        self.r_vel_furrow = r_vel_furrow
        self.r_vel_head = r_vel_head
        self.idle = True  # flag means that robot is at center
        self.done = False # flag means running to picker done
# Robots will go Manhanttan distance from center to picker go x first and go y
# p_pos.shape=(2,) p_pos[0] = x, p_pos[1] = y
# No preemption in the execution
    def run_to_picker(target_pos, dt):  
        delta_pos = target_pos - self.cur_pos
        self.idle = False
        # dist_pos = np.sum(np.abs(delta_pos))
        if np.abs(delta_pos[0]) > 0: 
            if dt*self.r_vel_head < np.abs(delta_pos[0]):
                self.cur_pos[0] += dt*self.r_vel_head*np.sign(delta_pos[0])
            else:
                self.cur_pos[0] = target_pos[0]
                t_left = dt-np.abs(delta_pos[0])/self.r_vel_head
                if np.abs(delta_pos[1]) > dt*self.r_vel_furrow:
                    self.cur_pos[1] += t_left*self.r_vel_furrow*np.sign(delta_pos[1]) 
                else:
                    self.cur_pos[1] = target_pos[1]
                    self.done = True
        else:
            if np.abs(delta_pos[1]) > dt*self.r_vel_furrow:
                self.cur_pos[1] += dt*self.r_vel_furrow*np.sign(delta_pos[1])
            else:
                self.cur_pos[1] = target_pos[1]
                self.done = True
        return self.cur_pos

    def run_back_center(target_pos, dt):
        delta_pos = target_pos - self.cur_pos
        # self.idle = False
        if np.abs(delta_pos[1]) > 0: 
            if dt*self.r_vel_furrow < np.abs(delta_pos[1]):
                self.cur_pos[1] += dt*self.r_vel_furrow*np.sign(delta_pos[0])
            else:
                self.cur_pos[1] = target_pos[1]
                t_left = dt-np.abs(delta_pos[1])/self.r_vel_furrow
                if np.abs(delta_pos[0]) > dt*self.r_vel_head:
                    self.cur_pos[0] += t_left*self.r_vel_head*np.sign(delta_pos[0]) 
                else:
                    self.cur_pos[0] = target_pos[1]
        else:
            if np.abs(delta_pos[1]) > dt*self.r_vel_furrow:
                self.cur_pos[1] += dt*self.r_vel_furrow*np.sign(delta_pos[1])
            else:
                self.cur_pos[1] = target_pos[1]
                self.done = False
                self.idle = True
        return self.cur_pos

In [173]:
# p_m_vel: moving velocity while picking
# p_w_vel: walking velocity while changing furrow
# p_pos: initial position of pickers
class Picker(object):
    def __init__(self, p_m_vel, p_w_vel, initial_p_pos, level, cart, time_of_day):
        # picking info
        self.cart = cart
        self.tod = time_of_day 
        self.level = level
        self.wt = 0
        self.cur_pos = initial_p_pos
        self.p_w_vel = p_w_vel
        self.p_m_vel = p_m_vel
        self.center = 0 # return center for the picker
        self.time_of_day = time_of_day
        self.wait_flag = False
        self.done = False # current furrow picking is done
        self.picking = True # flag of whether picking
        # for updating estimation finishing interval and std based on current instant
        self.ft_mean, self.ft_std = self._initial_estimate()
    def position_update(dt):
        if self.wait_flag:
            return self.cur_pos
        if not self.done:
            _picking_move(self,dt)
        else:
            _walk(self,dt)
        return self.cur_pos
    def update_wt(dt):
        # updating waiting time of pickers
        if self.wait_flag:
            self.wt += 1
        else:
            self.wt = 0
        return self.wt
    def mechanistic_predictive_model(self, delta_T):
        self.ft_mean -= delta_T
        if np.random.normal(self.ft_mean,self.ft_std) < 1:
            self.wait_flag = True
    def _picking_move(self, dt):
        # update position while pickers are picking
        if self.cur_pos[1] > y_head:
            self.cur_pos[1] -= dt*self.p_m_vel
        else:
            self.done = True
    def _walk(self, target_pos, dt):
        #update position of pickers while it is walking
        delta_pos = target_pos - self.cur_pos
        self.picking = False
        if np.abs(delta_pos[0]) > 0: 
            if dt*self.p_w_vel < np.abs(delta_pos[0]):
                self.cur_pos[0] += dt*self.p_w_vel*np.sign(delta_pos[0])
            else:
                self.cur_pos[0] = target_pos[0]
                t_left = dt-np.abs(delta_pos[0])/self.p_w_vel
                if np.abs(delta_pos[1]) > dt*self.p_w_vel:
                    self.cur_pos[1] += t_left*self.p_w_vel*np.sign(delta_pos[1]) 
                else:
                    self.cur_pos[1] = target_pos[1]
                    self.picking = True
        else:
            if np.abs(delta_pos[1]) > dt*self.p_w_vel:
                self.cur_pos[1] += dt*self.p_w_vel*np.sign(delta_pos[1])
            else:
                self.cur_pos[1] = target_pos[1]
                self.picking = True	

    def _initial_estimate(self):
        if self.level==0 and self.time_of_day==0 and self.cart==0:
            initial_ft_mean = 477.1
            initial_ft_std = 42.4
        return initial_ft_mean, initial_ft_std

In [194]:
class BerryPick():
    def __init__( self, pick_num, robot_num,      # field info 
                   p_m_vel, p_w_vel, p_pos, level, # picker info
                   r_pos,                          # robot info
                   time_of_day, cart, center):    # field info 
    # initialize constant values
        self.p_num = pick_num
        self.r_num = robot_num
        self.initial_ps_pos = p_pos
        self.p_m_vel = p_m_vel
        self.p_w_vel = p_w_vel
        self.level = level
        self.cart = cart
        self.tod = time_of_day
        self.initial_rs_pos = r_pos
        self.r_vel_furrow = 10 # inch/s
        self.r_vel_head = 20
        self.center = center
        # instance list of robots and pickers
        self.robots = []
        self.pickers = []
        time_steps = 0
        # initialize n pickers model with initial speed and initial variance
        self.assign = np.zeros((1,self.p_num))
        self.s_t =self._initialization()
        
        # inital assignment
        # r = np.ones(num_robots,dtype=int)
        # p = np.zeros(num_pickers-num_robots,dtype=int)
        # assign = np.concatenate(r,p)
        # self.initial_assign = np.random.permutation(assign)

    def reset(self):
        initial_state = _initialization()
        return initial_state
    # reset environment randomly
    def step(self,action):
#         return self.s_t, reward, self.done
        pass
    # execute one action
    def render():
        pass
    # render current state to picture
    def _initialization(self):
#         print self.assign
    # initialize all pickers and robots
        for i, m_vel, w_vel, pos, l, c, tod in zip(range(self.p_num), self.p_m_vel, self.p_w_vel, self.initial_ps_pos, self.level, self.cart, self.tod):
            p = Picker(m_vel, w_vel, pos, l, c, tod)
            self.pickers.append(p)
        # initialize m robots with inital positions veloctiy is a constant value
        for i, pos in zip(range(self.r_num), self.initial_rs_pos):
            r = Robot(pos, self.r_vel_furrow, self.r_vel_head)
            self.robots.append(r)
        # initial state
        initial_state = [] # ft_mean, ft_std, p_pos, p_wt, r_pos
        ft_mean, ft_std, ps_pos, ps_wt, rs_pos = [], [], [], [], []
        # obtain pickers and robot initial state
        for i in range(self.p_num):
            if i < self.r_num:
                r_pos = self.robots[i].cur_pos
            else:
                r_pos = np.array([0,0]) # compensate 0s

            mean = self.pickers[i].mechanistic_predictive_model(0)
            std  = self.pickers[i].mechanistic_predictive_model(0)
            pos = self.pickers[i].cur_pos
            wt = self.pickers[i].wt

            ft_mean.append(mean)
            ft_std.append(std)
            ps_pos.append(pos)
            ps_wt.append(wt)
            rs_pos.append(r_pos)

        ft_mean = np.array([ft_mean]) # 1*n
        ft_std = np.array([ft_std])   # 1*n
        ps_pos = np.array(ps_pos).T       # 2*n
        ps_wt = np.array([ps_wt])         # 1*n
        rs_pos = np.array(rs_pos).T     # 2*n
        center1=np.tile(self.center[0],(20,1))
        center2=np.tile(self.center[1],(20,1))
        centers = np.concatenate((center1,center2),axis=1).T
        print ft_mean.shape, ft_std.shape,ps_pos.shape,ps_wt.shape,rs_pos.shape,centers.shape, self.assign.shape
        initial_state = np.concatenate((ft_mean,ft_std,ps_pos,ps_wt,rs_pos,centers,self.assign),axis=0)

        return initial_state 

In [195]:
def field_initialization(picker_num, robot_num, y_head, furrow_width, berry_width, furrow_length):
    center_1 = [(furrow_width+berry_width)*picker_num/2, 0]
    center_2 = [-(furrow_width+berry_width)*picker_num/2, 0]
    center = np.array([center_1, center_2])
    r_initial_pos = []
    p_initial_pos = []
    for i in range(robot_num):
        r_initial_pos.append(center_1)
    r_initial_pos = np.asarray(r_initial_pos) # m*2, m1:r_x, m2:r_y

    for i in range(picker_num):
        x = furrow_width/2 + (furrow_width+berry_width)*i
        pos = [x, y_head + furrow_length]
        p_initial_pos.append(pos)
    p_initial_pos = np.asarray(p_initial_pos) # n*2, n1:p_x, n2:p_y
    return p_initial_pos, r_initial_pos, center

In [196]:
# fields information
origin = np.asarray([0,0])
furrow_width = 28
berry_width = 14
furrow_length = 200
picker_num = 20
robot_num = 4
y_head = 40 # y coordinate of furrow head
# pickers information
time_of_day = np.zeros(picker_num)# 0 is morining, 1 is afternoon
cart = np.zeros(picker_num)
# speed_level = np.concatenate((np.ones((10,1)), np.zeros((10,1))), axis=0)[:,0] # 0 is fast, 1 is slow, 10 fast, 10 slow
speed_level = np.zeros(picker_num)
# print speed_level
cart_style = 0
p_initial_pos, r_initial_pos, center = field_initialization(picker_num, robot_num, y_head, furrow_width, berry_width, furrow_length)
print p_initial_pos
print r_initial_pos 
print center

[[ 14 240]
 [ 56 240]
 [ 98 240]
 [140 240]
 [182 240]
 [224 240]
 [266 240]
 [308 240]
 [350 240]
 [392 240]
 [434 240]
 [476 240]
 [518 240]
 [560 240]
 [602 240]
 [644 240]
 [686 240]
 [728 240]
 [770 240]
 [812 240]]
[[420   0]
 [420   0]
 [420   0]
 [420   0]]
[[ 420    0]
 [-420    0]]


In [197]:
print center.T
center1=np.tile(center[0],(20,1))
center2=np.tile(center[1],(20,1))
np.concatenate((center1,center2),axis=1)

[[ 420 -420]
 [   0    0]]


array([[ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0],
       [ 420,    0, -420,    0]])

In [198]:
p_m_vel = np.ones(picker_num) 
p_w_vel = np.ones(picker_num)*10

BP_env = BerryPick(picker_num, robot_num,     
             p_m_vel, p_w_vel, p_initial_pos, speed_level, 
             r_initial_pos,                          
             time_of_day, cart, center)

(1, 20) (1, 20) (2, 20) (1, 20) (2, 20) (4, 20) (1, 20)


In [203]:
BP_env.pickers[0].mechanistic_predictive_model(1)
BP_env.pickers[0].ft_mean

476.1